In [12]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
import json

In [13]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)

In [14]:
dataset = []
count = 0
for l in parse("review-Hawaii_10.json.gz"):
  dataset.append(l)


In [15]:
len(dataset)

1504347